# Document split experiment
@author Hieu
@date 30/12/2023

1. Tools
- FAISS
- LaBSE

In [3]:
import pandas as pd
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings, HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
from getpass import getpass
import sys
sys.path.append("../")
from src.config import Configuration
conf = Configuration()

# HUGGING_FACE_KEY = getpass("Hugging face key: ")
# HUGGING_FACE_KEY = open("../.hg_key").read()
db_df = pd.read_csv("../data/majors_info.csv")
db_df.shape

(40, 3)

In [4]:
# LaBSE
# MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
# MODEL = "keepitreal/vietnamese-sbert"

MODEL = "sentence-transformers/LaBSE"
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=conf.load_hg_token(3),
    model_name=MODEL)


# Load Data

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"],
    chunk_size=1600,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

In [5]:
db = []
dist = []
for i, data in db_df.iterrows():
    doc = text_splitter.create_documents([data['Description']])
    dist.append(len(doc))
    [d.metadata.update({"doc_title": data['Major']}) for d in doc]
    db.extend(doc)

In [6]:
from collections import Counter
Counter(dist)

Counter({3: 22, 2: 17, 4: 1})

In [7]:
len(db)

104

In [5]:
faiss_db = FAISS.from_documents(db, embeddings)
len(db)

113

# Learn

In [31]:
import numpy as np

a = embeddings.embed_query("Tôi muốn học về lập trình")
test_docs = embeddings.embed_documents(\
    ["kỹ thuật xây dựng", "xây dựng",\
     "phần mềm", "khoa học máy tính", "python", "java",\
     "Phần mềm vẽ kỹ thuật cho thiết kế"])

npa = np.array(a)
for b in test_docs:
    npb = np.array(b)
    distance = np.linalg.norm(npa - npb)
    print(distance)

1.0123114538765887
1.1077420049467213
1.095200175904101
0.9544046579578791
1.2669979125528543
1.163662254111237
0.973616333800583


# RUN

In [11]:
query = "Tôi muốn học về lập trình"
res = faiss_db.similarity_search_with_score(query)
print("Query: "+query)
a = [print(r[0].metadata, "- euclid: ", r[1]) for r in res]

Query: Tôi muốn học về lập trình
{'doc_title': 'Kỹ thuật phần mềm'} - euclid:  0.86084354
{'doc_title': 'Kiến trúc'} - euclid:  0.8648312
{'doc_title': 'Kỹ thuật phần mềm'} - euclid:  0.87316966
{'doc_title': 'Kỹ thuật điện tử - viễn thông'} - euclid:  0.9010457


In [12]:
query = "Hãy tư vấn ngành công nghệ thông tin"
res = faiss_db.similarity_search_with_score(query)
print("Query: "+query)
a = [print(r[0].metadata, "- euclid: ", r[1]) for r in res]

Query: Hãy tư vấn ngành công nghệ thông tin
{'doc_title': 'Khoa học máy tính'} - euclid:  1.101366
{'doc_title': 'Khoa học máy tính'} - euclid:  1.1319034
{'doc_title': 'Xã hội học'} - euclid:  1.1676772
{'doc_title': 'Kỹ thuật phần mềm'} - euclid:  1.1724229


In [13]:
query = "Tôi học giỏi môn toán nhất, nếu vậy thì nên theo ngành nào"
res = faiss_db.similarity_search_with_score(query)
a = [print(r[0].metadata, "- euclid: ", r[1]) for r in res]

{'doc_title': 'Toán ứng dụng'} - euclid:  1.0212147
{'doc_title': 'Thống kê'} - euclid:  1.0324715
{'doc_title': 'Quản trị kinh doanh – Chuyên ngành Quản trị nguồn nhân lực'} - euclid:  1.0586259
{'doc_title': 'Thống kê'} - euclid:  1.0705578


In [14]:
query = "Tôi học giỏi môn toán nhất"
res = faiss_db.similarity_search_with_score(query)
a = [print(r[0].metadata, "- euclid: ", r[1]) for r in res]

{'doc_title': 'Toán ứng dụng'} - euclid:  0.9985891
{'doc_title': 'Thống kê'} - euclid:  1.0013721
{'doc_title': 'Thống kê'} - euclid:  1.006902
{'doc_title': 'Quản trị kinh doanh – Chuyên ngành Quản trị nguồn nhân lực'} - euclid:  1.0236031


In [15]:
query = "Toán học"
res = faiss_db.similarity_search_with_score(query)
a = [print(r[0].metadata, "- euclid: ", r[1]) for r in res]

{'doc_title': 'Toán ứng dụng'} - euclid:  1.0613945
{'doc_title': 'Thống kê'} - euclid:  1.1189573
{'doc_title': 'Thống kê'} - euclid:  1.1418173
{'doc_title': 'Khoa học máy tính'} - euclid:  1.1636293


In [16]:
query = "Tôi có định hướng làm về AI thì nên theo học ngành nào?"
res = faiss_db.similarity_search_with_score(query)
a = [print(r[0].metadata, "- euclid: ", r[1]) for r in res]

{'doc_title': 'Kỹ thuật phần mềm'} - euclid:  1.0318902
{'doc_title': 'Khoa học máy tính'} - euclid:  1.0495627
{'doc_title': 'Việt Nam học – Chuyên ngành Du lịch và lữ hành'} - euclid:  1.0708798
{'doc_title': 'Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch'} - euclid:  1.0749915


In [17]:
query = "AI"
res = faiss_db.similarity_search_with_score(query)
a = [print(r[0].metadata, "- euclid: ", r[1]) for r in res]

{'doc_title': 'Khoa học máy tính'} - euclid:  1.4101307
{'doc_title': 'Khoa học máy tính'} - euclid:  1.4656203
{'doc_title': 'Toán ứng dụng'} - euclid:  1.4928157
{'doc_title': 'Kỹ thuật phần mềm'} - euclid:  1.495049


In [18]:
query = "Toán học và vẽ"
res = faiss_db.similarity_search_with_score(query)
a = [print(r[0].metadata, "- euclid: ", r[1]) for r in res]

{'doc_title': 'Kiến trúc'} - euclid:  0.98648745
{'doc_title': 'Thiết kế nội thất'} - euclid:  0.98955333
{'doc_title': 'Thiết kế đồ họa'} - euclid:  0.99542654
{'doc_title': 'Toán ứng dụng'} - euclid:  0.9974606


In [23]:
query = "Tôi muốn cống hiến cho xã hội"
res = faiss_db.similarity_search_with_score(query)
a = [print(r[0].metadata, "- euclid: ", r[1]) for r in res]

{'doc_title': 'Công tác xã hội'} - euclid:  0.88076824
{'doc_title': 'Xã hội học'} - euclid:  0.9183835
{'doc_title': 'Công tác xã hội'} - euclid:  0.9257453
{'doc_title': 'Việt Nam học - Chuyên ngành Việt ngữ học và văn hóa xã hội Việt Nam'} - euclid:  1.018566


In [25]:
query = "đảng viên"
res = faiss_db.similarity_search_with_score(query)
a = [print(r[0].metadata, "- euclid: ", r[1]) for r in res]

{'doc_title': 'Luật'} - euclid:  1.42275
{'doc_title': 'Công tác xã hội'} - euclid:  1.4256456
{'doc_title': 'Công tác xã hội'} - euclid:  1.4261575
{'doc_title': 'Thiết kế nội thất'} - euclid:  1.4427919


In [ ]:
https://github.com/Searchium-ai/hybrid-search

# TEST

In [5]:
PATH = '../data/test_major'

pv_qas = pd.read_csv(f"{PATH}/private_test_case.csv")
qas = pd.read_csv(f"{PATH}/public_test_case.csv")

In [6]:
es = ElasticsearchStore.from_documents(db, embedding=embeddings,
                          es_connection=conf.load_elasticsearch_connection(),
                          index_name="test-db",
                          distance_strategy="EUCLIDEAN_DISTANCE")

NameError: name 'db' is not defined

In [7]:
es = ElasticsearchStore(embedding=embeddings,
                          es_connection=conf.load_elasticsearch_connection(),
                          index_name="test-db",
                          distance_strategy="EUCLIDEAN_DISTANCE")

# Match 1

In [6]:

for id, row in qas.iterrows():
    query = row['public question']
    res = faiss_db.similarity_search_with_score(query)
    target_id = res[0][0].metadata['id']
    if(int(target_id) == int(row['doc_id'])):
        correct += 1
correct/qas.shape[0]

0.6858974358974359

# Benchmark LaBSE

In [8]:
def test_major(retiever, limit):
    # match top 2
    correct = 0
    total = 0
    for id, row in qas.iterrows():
        query = row['public question']
        total += 1
        try:
            res = retiever.similarity_search_with_score(query)
            for idx, r in enumerate(res):
                target_id = r[0].metadata['id']
                if(int(target_id) == int(row['doc_id'])):
                    correct += 1
                    break
                if idx+1 == limit:
                    break
        except:
            print(query)
    return correct/total

def test_major_private(retiever, limit):
    # match top 2
    correct = 0
    total = 0
    for id, row in pv_qas.iterrows():
        query = row['private question']
        total += 1
        try:
            res = retiever.similarity_search_with_score(query)
            for idx, r in enumerate(res):
                target_id = r[0].metadata['id']
                if(int(target_id) == int(row['doc_id'])):
                    correct += 1
                    break
                if idx+1 == limit:
                    break
        except Error:
            print(e)
            print(query)
            break
    return correct/total

In [6]:
b = test_major(faiss_db, 1)
b

0.6858974358974359

In [7]:
b = test_major(faiss_db, 2)
b

0.8269230769230769

In [9]:
b = test_major_private(es, 1)
b

NameError: name 'e' is not defined